In [2]:
import spacy
import numpy as np


In [4]:
nlp = spacy.load("en_core_web_lg")

In [5]:
nlp

In [6]:

# Gor reproducing same results during mutiple run
s = 999
np.random.seed(s)
spacy.util.fix_random_seed(s)

# if Training with GPU also
# cupy.random.seed(s)


In [16]:

import pandas as pd
dpath = '2.json'
#df = pd.read_json(dpath, sep=',')
#words  = df.word.values
#ents = df.label.values
#text = ' '.join(words)

## Adding new annotations

In [8]:
add_ents = ['Case_References'] # 

# Create a pipe if it does not exist
# Piplines in pretrained model: tagger, parser, ner create new if blank model is to be trained using `spacy.blank('en')`
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner") # "architecture": "ensemble" simple_cnn ensemble, bow # https://spacy.io/api/annotation
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe("ner")

prev_ents = ner.move_names
print('[Existing Entities] = ', ner.move_names)

for ent in add_ents:
    ner.add_label(ent)
    
new_ents = ner.move_names
# print('\n[All Entities] = ', ner.move_names)

print('\n\n[New Entities] = ', list(set(new_ents) - set(prev_ents)))

[Existing Entities] =  ['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O']


[New Entities] =  ['B-Case_References', 'U-Case_References', 'L-Case_References', 'I-Case_References']


# Creating a dataset

In [17]:
#### Create Dataset
from spacy.gold import GoldParse
doc = nlp.make_doc(text)
g = GoldParse(doc, entities=ents)

# Add examples as avaialble or needed
X = [doc, doc]
Y = [g, g]

ModuleNotFoundError: No module named 'spacy.gold'

In [18]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
print(f'[OtherPipes] = {other_pipes} will be disabled')

[OtherPipes] = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'] will be disabled


In [19]:
model = None # Since we training a fresh model not a saved model
n_iter = 20
with nlp.disable_pipes(*other_pipes):  # only train ner
    # optimizer = nlp.begin_training()
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    for i in range(n_iter):
        losses = {}
        nlp.update(X, Y,  sgd=optimizer, drop=0.0, losses=losses)
        # nlp.entity.update(d, g)
        print("Losses", losses)

ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.

In [20]:
test_text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(test_text)
# print("[Entities] in '%s'" % test_text, '\n\n')
for ent in doc.ents:
    print(ent.text, ent.label_)

Sebastian Thrun PERSON
2007 DATE
American NORP
Thrun ORG
earlier this week DATE
